# Obdelava velepodatkov z GPE

Velepodatke lahko obdelujemo na različnih procesnih enotah. Najpogosteje se to izvaja na CPE (centralna procesna enota - procesor), lahko pa obdelavo izvajamo tudi na GPE (grafična procesna enota - grafična kartica). To je še posebej koristno kadar učimo modele umetne inteligence, saj lahko podatke prenesemo na GPE, jih tam obdelamo, nato pa jih pošljemo skozi operacije, ki so del algoritma umetne inteligence (razni klasifikatorji in nevronske mreže). S tem pohitrimo celoten delovni tok, saj zmanjšamo čas prenosa podatkov iz ene procesne enote na drugo. V tej enoti se boste seznanili z obdelavo velepodatkov na GPE. Spoznali boste knjižnico ``cuDF``, ki je del ogrodja NVIDIA RAPIDS. Knjižnica ``cuDF`` deluje z grafičnimi procesnimi enotami NVIDIA, ki imajo CUDA jedra. V primeru, da na vašem sistemu nimate grafične kartice NVIDIA, je priporočljivo, da ta Jupyter zvezek izvajate v okolju Google Colab ali Kaggle Notebook.

## Prenos podatkovne zbirke

Najprej bomo prenesli podatkovno podatkovno zbirko, ki jo bomo najprej obdelali s funkcijami knjižnice Pandas, nato pa še s funkcijami knjižnice cuDF. Podatkovna zbirka je namenjena analizi sentimenta v tvitih in vsebuje imena uporabniških računov, tvite in oznake sentimenta. Zbirko bomo najprej dopolnili z novimi stolpci potem pa bomo nad temi podatki izvajali različne operacije in poizvedbe.

In [1]:
# prenos podatkovne zbirke
!wget https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

--2024-05-27 07:52:01--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  50.1MB/s    in 1.5s    

2024-05-27 07:52:03 (50.1 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]



In [2]:
# ekstrakcija podatkovne zbirke
from zipfile import ZipFile

archive = ZipFile('trainingandtestdata.zip')
archive.extractall()

## Obdelava brez GPE (Pandas)

Najprej vključimo knjižnico Pandas in naložimo podatkovno zbirko, nato pa začnimo z obdelavo. V spodnji celici bomo preverili ali Pandas uporablja GPE ali CPE. Dodali bomo tudi pomožno funkcijo za izpis velikosti podatkovne zbirke.

In [3]:
import pandas as pd
import humanize

def get_size(df):
    return humanize.intcomma(len(df))

# preverimo ali Pandas uporablja CPE ali GPE
pd

<module 'pandas' from '/usr/local/lib/python3.10/dist-packages/pandas/__init__.py'>

V izpisu vidimo, da se uporablja modul Pandas, ki ne deluje pospešeno (tj. brez GPE). Knjižnica Pandas bo v nadaljevanju uporabljala CPE za obdelavo podatkov. Naložimo podatkovno zbirko in izpišimo prvih 5 vrstic.

In [4]:
%time df = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, encoding_errors='replace').drop(columns=[1, 2, 3])
df.columns = ['label', 'user', 'tweet'] # preimenujmo stolpce

print(f'Dataset rows: {get_size(df)}') # 1,6 mio vrstic
df.head() # izpišimo prvih 5 vrstic

CPU times: user 4.77 s, sys: 550 ms, total: 5.32 s
Wall time: 5.74 s
Dataset rows: 1,600,000


,label,user,tweet
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,scotthamilton,is upset that he can't update his Facebook by ...
2,0,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,ElleCTF,my whole body feels itchy and like its on fire
4,0,Karoli,"@nationwideclass no, it's not behaving at all...."


Nalaganje 1,6 mio vrstic je trajalo 5,74s. Sami premislite kako bi preverili porabo pomnilnika. V nadaljevanju bomo izvedli nekaj dodatnih operacij nad podatkovno zbirko in ustvarili nove stolpce po katerih bomo izvajali poizvedbe.

Najprej bomo iz tvitov izluščili vse omembe uporabnikov. To bomo storili z regularnim izrazom, kjer bomo iskali začetni znak `@`. Vse najdene omembe bomo shranili v stolpec `mentions`. Podobno bomo iskali vse oznake oz. ključnike *(ang. tags)*. Tudi pri tem bomo uporabili regularni izraz, kjer bomo iskali začetni znak `#`. Ustvarili bomo nov stolpec `tags` kjer bomo shranili rezultate tega koraka. Nazadnje bomo izluščili še vse povezave. Še enkrat bomo uporabili regularni izraz, kjer bomo iskali začetni niz znakov `http` in rezultate shranili v nov stolpec `links`.

Nato bomo vse črke v tvitih pretvorili v male črke, vse pojavitve omemb, oznak in povezav pa bomo nadomestili z nizi `USER`, `TAG` in `LINK`. Nazadnje bomo ustvarili še 3 nove stolpce: `mentions_count`, `tags_count` in `links_count`. Sem bomo shranili števila izluščenih omemb, oznak in povezav. Na koncu bomo izpisali prvih 5 vrstic obdelane podatkovne zbirke.

In [5]:
%%time
# luščenje omemb, oznak in povezav
df['mentions'] = df['tweet'].str.findall('@\S+')
df['tags'] = df['tweet'].str.findall('#\S+')
df['links'] = df['tweet'].str.findall('http\S+')

df['tweet'] = df['tweet'].str.lower() # pretvorba v male črke

# zamenjava omemb, oznak in povezav z oznakami USER, TAG in LINK
df['tweet'] = df['tweet'].str.replace('@\S+', 'USER')
df['tweet'] = df['tweet'].str.replace('#\S+', 'TAG')
df['tweet'] = df['tweet'].str.replace('http\S+', 'LINK')

# ustvarjanje novih stolpcev s številom omemb, oznak in povezav
df['mentions_count'] = df['mentions'].apply(lambda x: len(x))
df['tags_count'] = df['tags'].apply(lambda x: len(x))
df['links_count'] = df['links'].apply(lambda x: len(x))

df.head() # izpišimo prvih 5 vrstic

CPU times: user 8.25 s, sys: 446 ms, total: 8.7 s
Wall time: 8.71 s


,label,user,tweet,mentions,tags,links,mentions_count,tags_count,links_count
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",[@switchfoot],[],[http://twitpic.com/2y1zl],1,0,1
1,0,scotthamilton,is upset that he can't update his facebook by ...,[],[],[],0,0,0
2,0,mattycus,@kenichan i dived many times for the ball. man...,[@Kenichan],[],[],1,0,0
3,0,ElleCTF,my whole body feels itchy and like its on fire,[],[],[],0,0,0
4,0,Karoli,"@nationwideclass no, it's not behaving at all....",[@nationwideclass],[],[],1,0,0


Zgornje operacije so trajale 8,71s. Imamo obdelano podatkovno zbirko zato se lotimo poizvedb. Želimo doseči izpis, kjer bo razvidno grupiranje po tipu sentimenta (0 ali 4) za vsakega uporabnika. Pri tem nas zanima število tvitov vsakega uporabnika. Najprej grupirajmo podatke po stolpcih `label` in `user`, nato pa pridobimo število tvitov.

In [6]:
%time df.groupby(['label', 'user']).size()

CPU times: user 2.4 s, sys: 60.1 ms, total: 2.46 s
Wall time: 2.51 s


label  user         
0      000catnap000     1
       006jazzy         1
       007LouiseOB      1
       007_Chris_007    3
       007buddha        1
                       ..
4      zzzandra         2
       zzzaney          1
       zzztar           2
       zzzum            1
       zzzunzinnn       3
Length: 792240, dtype: int64

Naša poizvedba je trajala 2,51s. Poskusimo z novo poizvedbo, kjer nas zanima število tvitov po uporabnikih, pri tem pa želimo dobiti prvih 10 uporabnikov z največ tviti.

In [7]:
%%time
(df
 .groupby('user')
 .size()
 .sort_values(ascending=False)
 .head(10)
)

CPU times: user 1.51 s, sys: 42.1 ms, total: 1.55 s
Wall time: 1.56 s


user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
mcraddictal        276
tsarnick           248
what_bugs_u        246
Karen230683        238
DarkPiano          236
dtype: int64

Poizvedba je trajala 1,51s. Izvedimo še poizvedbo s katero želimo dobiti prvih 10 tvitov, ki imajo več kot eno oznako (tag) in tip sentimenta enak 0. Pri tem uredimo te tvite v padajočem vrstnem redu po številu oznak.

In [8]:
%%time
(df
 .query("`label` == 0 & `tags_count` > 1")
 .sort_values('tags_count', ascending=False)
 .head(10)
)

CPU times: user 17.4 ms, sys: 10.9 ms, total: 28.3 ms
Wall time: 24.6 ms


,label,user,tweet,mentions,tags,links,mentions_count,tags_count,links_count
715830,0,jonafied26,#bts is fallin!! #bts #bts #bts #bts #bts #b...,[],"[#BTS, #BTS, #BTS, #BTS, #BTS, #BTS, #BTS, #BT...",[],0,24,0
361995,0,findthesun,#seb-day #seb-day #seb-day #seb-day #seb-day #...,[],"[#seb-day, #seb-day, #seb-day, #seb-day, #seb-...",[],0,15,0
229001,0,quintosential,#ontd is making me tired #ontd #ontd #ontd #...,[],"[#ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ON...",[],0,15,0
368534,0,mahsimpleplan,mimimimi #seb-day #seb-day #seb-day #seb-day ...,[],"[#seb-day, #seb-day, #seb-day, #seb-day, #seb-...",[],0,14,0
356824,0,caroldonada,@flavianalin ainda nao #seb-day #seb-day #seb...,[@flavianalin],"[#seb-day, #seb-day, #seb-day, #seb-day, #seb-...",[],1,12,0
712592,0,Ronjaw,oh no i dont like camilla #bts #bts #bts #bt...,[],"[#bts, #bts, #bts, #bts, #bts, #bts, #bts, #bt...",[],0,12,0
540173,0,cutthroatpixie,#imisscath #imisscath #imisscath #imisscath #...,[],"[#IMISSCATH, #IMISSCATH, #IMISSCATH, #IMISSCAT...",[],0,12,0
226894,0,mattchew03,watching old episodes of &quot;the sopranos&qu...,[],"[#ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ON...",[],0,12,0
540159,0,cutthroatpixie,#imisscath #imisscath #imisscath #imisscath #...,[],"[#IMISSCATH, #IMISSCATH, #IMISSCATH, #IMISSCAT...",[],0,12,0
338202,0,bushido02,@teapartynews yea..i know...momentary lapse of...,[@teapartynews],"[#tcot, #ac2c, #hhrs, #912, #sgp, #tlot, #gwot...",[],1,11,0


Ta poizvedba je trajala 24,6ms. Poskusimo z novo poizvedbo, kjer nas zanima prvih 25 tvitov uporabnika 'lost_dog', ki imajo tip sentimenta enak 0, hkrati pa želimo črke teh tvitov pretvoriti v velike črke. Nazadnje še izvedimo urejanje v obratnem abecednem redu in ponastavimo indeksiranje.

In [9]:
%%time
(df
 .query("`label` == 0 & `user` == 'lost_dog'")['tweet']
 .apply(lambda x: x.upper())
 .sort_values(ascending=False)
 .reset_index()
 .head(25)
)

CPU times: user 98.1 ms, sys: 68 µs, total: 98.2 ms
Wall time: 96.8 ms


,index,tweet
0,139106,@_MARKWOOD I AM LOST. PLEASE HELP ME FIND A GO...
1,683677,@_INTRICACY I AM LOST. PLEASE HELP ME FIND A G...
2,46919,@ZUPPAHOLIC I AM LOST. PLEASE HELP ME FIND A G...
3,493775,@ZUBBYTUBBY I AM LOST. PLEASE HELP ME FIND A G...
4,240948,@ZOUNA I AM LOST. PLEASE HELP ME FIND A GOOD H...
5,270830,@ZENSHADOW I AM LOST. PLEASE HELP ME FIND A GO...
6,535890,@ZACHARYGRESSER I AM LOST. PLEASE HELP ME FIND...
7,301709,@YOURDOGTWEETS I AM LOST. PLEASE HELP ME FIND ...
8,479739,@YOURAGENTINAZ I AM LOST. PLEASE HELP ME FIND ...
9,661505,@YONITIN1 I AM LOST. PLEASE HELP ME FIND A GOO...


Ta poizvedba je trajala 96,8ms. Preizkusili smo več uporabnih funkcij knjižnice Pandas, ki pa so se izvajale na CPE. V nadaljevanju bomo ponovili vse operacije, vendar bomo tokrat uporabili knjižnico cuDF.

## Obdelava z GPE (Pandas + cuDF)

Knjižnica cuDF je zelo enostavna za uporabo, saj so poizvedbe enake kot, če bi uporabljali samo Pandas. Prav tako niso potrebne kakšne druge posebnosti, razen namestitve paketa knjižnice cuDF. Knjižnica Pandas podpira pospeševanje s knjižnico cuDF tako, da lahko to enostavno vključimo in s tem omogočimo izvajanje operacij na GPE. Pri tem je potrebno povdariti, da je izvajanje nekaterih operacij na GPE omejeno in se zaradi tega avtomatsko izvajajo na CPE. Seznam podprtih operacij Pandas in omejitev v cuDF je naveden v [dokumentaciji cuDF](https://docs.rapids.ai/api/cudf/stable/user_guide/pandascompat/).

Preverimo kako se obnese pospeševanje operacij Pandas s cuDF. Najprej bomo vključili pospeševanje s cuDF nato pa se prepričali, da knjižnica Pandas deluje v načinu s pospeševanjem.

In [10]:
# POMEMBNO!
# Pred uporabo cuDF moramo namestiti knjižnico cuDF ali pa celoten nabor knjižnic v ogrodju NVIDIA RAPIDS.
# Če delate v okolju Google Colab, vam tega ni potrebno storiti, saj je cuDF že nameščen.
# Sicer lahko namestite cuDF ali NVIDIA RAPIDS s pomočjo navodil na spletni strani: https://docs.rapids.ai/install

# Ukaz %load_ext cudf.pandas deluje samo v zvezku Jupyter.
# Če boste to kodo izvajali ločeno s programom v programskem jeziku Python, je potrebno ob zagonu programa uporabiti zastavico '-m cudf.pandas'
# Primer: python ./moj_program.py -m cudf.pandas

# vključimo pospeševanje s cuDF
%load_ext cudf.pandas
import pandas as pd # ponovno naložimo Pandas - tokrat v načinu s pospeševanjem

# preverimo ali Pandas uporablja CPE ali GPE
pd

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

V zgornjem izpisu zdaj vidimo, da Pandas deluje v načinu s pospeševanjem, pri tem pa uporablja knjižnico cuDF. Izvedimo enake operacije kot smo jih izvedli v sekciji brez pospeševanja.

In [11]:
%time gdf = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, encoding_errors='replace').drop(columns=[1, 2, 3])
gdf.columns = ['label', 'user', 'tweet'] # preimenujmo stolpce

print(f'Dataset rows: {get_size(gdf)}') # 1,6 mio vrstic
gdf.head() # izpišimo prvih 5 vrstic

CPU times: user 4.86 s, sys: 851 ms, total: 5.71 s
Wall time: 5.83 s
Dataset rows: 1,600,000


,label,user,tweet
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,scotthamilton,is upset that he can't update his Facebook by ...
2,0,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,ElleCTF,my whole body feels itchy and like its on fire
4,0,Karoli,"@nationwideclass no, it's not behaving at all...."


Nalaganje 1,6 mio vrstic je trajalo 5,83s kar je praktično enako kot v načinu brez pospeševanja. Izvedimo obdelavo podatkov in preostale poizvedbe, pri tem pa se osredotočimo na hitrost izvajanja. Za to bomo uporabljali ukaza %time in %%time.

In [12]:
%%time

# luščenje omemb, oznak in povezav
gdf['mentions'] = gdf['tweet'].str.findall('@\S+')
gdf['tags'] = gdf['tweet'].str.findall('#\S+')
gdf['links'] = gdf['tweet'].str.findall('http\S+')

gdf['tweet'] = gdf['tweet'].str.lower() # pretvorba v male črke

# zamenjava omemb, oznak in povezav z oznakami USER, TAG in LINK
gdf['tweet'] = gdf['tweet'].str.replace('@\S+', 'USER')
gdf['tweet'] = gdf['tweet'].str.replace('#\S+', 'TAG')
gdf['tweet'] = gdf['tweet'].str.replace('http\S+', 'LINK')

# ustvarjanje novih stolpcev s številom omemb, oznak in povezav
gdf['mentions_count'] = gdf['mentions'].apply(lambda x: len(x))
gdf['tags_count'] = gdf['tags'].apply(lambda x: len(x))
gdf['links_count'] = gdf['links'].apply(lambda x: len(x))

gdf.head() # izpišimo prvih 5 vrstic

CPU times: user 12.8 s, sys: 340 ms, total: 13.2 s
Wall time: 13.3 s


,label,user,tweet,mentions,tags,links,mentions_count,tags_count,links_count
0,0,_TheSpecialOne_,"USER LINK - awww, that's a bummer. you should...",[@switchfoot],[],[http://twitpic.com/2y1zl],1,0,1
1,0,scotthamilton,is upset that he can't update his facebook by ...,[],[],[],0,0,0
2,0,mattycus,USER i dived many times for the ball. managed ...,[@Kenichan],[],[],1,0,0
3,0,ElleCTF,my whole body feels itchy and like its on fire,[],[],[],0,0,0
4,0,Karoli,"USER no, it's not behaving at all. i'm mad. wh...",[@nationwideclass],[],[],1,0,0


Zgornje operacije so trajale 13,3s kar je počasneje v primerjavi z izvedbo v načinu brez pospeševanja. Obdelava na GPE v tem primeru vzame približno 4,5 sekunde dalj časa kot na CPE. Pri tem je potrebno razumeti, da izvajamo operacije nad nizi znakov - tip podatka, za katerega GPE navadno niso optimizirane. Zaključimo lahko, da bi v praktični situaciji imelo več smisla, da obdelavo podatkov naredimo na CPE, šele nato pa izvajamo poizvedbe na GPE. Lotimo se poizvedb.

In [13]:
%%time
gdf.groupby(['label', 'user']).size()

CPU times: user 129 ms, sys: 26.8 ms, total: 155 ms
Wall time: 288 ms


label  user         
0      000catnap000     1
       006jazzy         1
       007LouiseOB      1
       007_Chris_007    3
       007buddha        1
                       ..
4      zzzandra         2
       zzzaney          1
       zzztar           2
       zzzum            1
       zzzunzinnn       3
Length: 792240, dtype: int64

Poizvedba je trajala 288ms kar je bistveno hitreje v primerjavi z izvedbo v načinu brez pospeševanja (2,51s). Jasno vidimo prednost izvajanja poizvedb na GPE, saj smo dosegli kar 8,7-kratno pohitritev izvajanja te poizvedbe! Poskusimo še s poizvedbo z več veriženimi operacijami.

In [14]:
%%time
(gdf
 .groupby('user')
 .size()
 .sort_values(ascending=False)
 .head(10)
)

CPU times: user 74.3 ms, sys: 30.9 ms, total: 105 ms
Wall time: 170 ms


user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
mcraddictal        276
tsarnick           248
what_bugs_u        246
Karen230683        238
DarkPiano          236
dtype: int64

Poizvedba je trajala 170ms kar je ponovno bistveno hitreje v primerjavi z izvedbo v načinu brez pospeševanja (1,56s). Tudi tukaj smo dosegli veliko pohitritev (9,1x).

In [15]:
%%time
(df
 .query("`label` == 0 & `tags_count` > 1")
 .sort_values('tags_count', ascending=False)
 .head(10)
)

CPU times: user 21.8 ms, sys: 12.8 ms, total: 34.6 ms
Wall time: 40.2 ms


,label,user,tweet,mentions,tags,links,mentions_count,tags_count,links_count
715830,0,jonafied26,#bts is fallin!! #bts #bts #bts #bts #bts #b...,[],"[#BTS, #BTS, #BTS, #BTS, #BTS, #BTS, #BTS, #BT...",[],0,24,0
361995,0,findthesun,#seb-day #seb-day #seb-day #seb-day #seb-day #...,[],"[#seb-day, #seb-day, #seb-day, #seb-day, #seb-...",[],0,15,0
229001,0,quintosential,#ontd is making me tired #ontd #ontd #ontd #...,[],"[#ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ON...",[],0,15,0
368534,0,mahsimpleplan,mimimimi #seb-day #seb-day #seb-day #seb-day ...,[],"[#seb-day, #seb-day, #seb-day, #seb-day, #seb-...",[],0,14,0
356824,0,caroldonada,@flavianalin ainda nao #seb-day #seb-day #seb...,[@flavianalin],"[#seb-day, #seb-day, #seb-day, #seb-day, #seb-...",[],1,12,0
712592,0,Ronjaw,oh no i dont like camilla #bts #bts #bts #bt...,[],"[#bts, #bts, #bts, #bts, #bts, #bts, #bts, #bt...",[],0,12,0
540173,0,cutthroatpixie,#imisscath #imisscath #imisscath #imisscath #...,[],"[#IMISSCATH, #IMISSCATH, #IMISSCATH, #IMISSCAT...",[],0,12,0
226894,0,mattchew03,watching old episodes of &quot;the sopranos&qu...,[],"[#ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ONTD, #ON...",[],0,12,0
540159,0,cutthroatpixie,#imisscath #imisscath #imisscath #imisscath #...,[],"[#IMISSCATH, #IMISSCATH, #IMISSCATH, #IMISSCAT...",[],0,12,0
338202,0,bushido02,@teapartynews yea..i know...momentary lapse of...,[@teapartynews],"[#tcot, #ac2c, #hhrs, #912, #sgp, #tlot, #gwot...",[],1,11,0


Ta poizvedba je trajala 40ms kar je malenkost počasneje v primerjavi z izvedbo v načinu brez pospeševanja (24,6ms). Uporabljamo funkcijo ``.query()``, ki je sicer optimizirana za izvedbo na CPE. Vidimo torej, da je hitrost izvedbe na GPE zelo odvisna od poizvedbe, tipa podatkov po katerih poizvedujemo in funkcij, ki jih pri tem uporabljamo.

In [16]:
%%time
(gdf
 .query("`label` == 0 & `user` == 'lost_dog'")['tweet']
 .apply(lambda x: x.upper())
 .sort_values(ascending=False)
 .reset_index()
 .head(25)
)

CPU times: user 15 s, sys: 1.01 s, total: 16 s
Wall time: 16.6 s


,index,tweet
0,43935,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
1,45574,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
2,46919,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
3,47949,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
4,50572,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
5,50854,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
6,55096,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
7,55656,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
8,60808,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
9,64297,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.


Ta poizvedba traja zelo dolgo - kar 16,6 sekund v primerjavi z zgolj 96,8ms z izvajanjem brez pospeševanja! Zakaj pride do tega? Premislite kaj bi lahko bil vzrok. Kako je mogoče, da dobimo tako porazen čas izvajanja?

![image](img/hmm.jpg)

### Profiliranje

Knjižnica cuDF nam nudi tudi nekaj uporabnih funkcionalnosti za raziskovanje takšnih problemov kot je ta. Našo kodo lahko profiliramo z vgrajenimi orodji. Poskusimo še enkrat zagnati enako poizvedbo, vendar jo tokrat profilirajmo. Tako bomo dobili boljši vpogled v izvajanje poizvedbe.

Profiliranje vključimo z ukazom ``%%cudf.pandas.profile`` kateremu sledi naša koda s poizvedbo.

In [17]:
%%cudf.pandas.profile
(gdf
 .query("`label` == 0 & `user` == 'lost_dog'")['tweet']
 .apply(lambda x: x.upper())
 .sort_values(ascending=False)
 .reset_index()
 .head(25)
)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 97, in __enter__
    sys.settrace(self._tracefunc)



,index,tweet
0,43935,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
1,45574,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
2,46919,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
3,47949,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
4,50572,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
5,50854,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
6,55096,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
7,55656,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
8,60808,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.
9,64297,USER I AM LOST. PLEASE HELP ME FIND A GOOD HOME.



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 116, in __exit__
    sys.settrace(self._oldtrace)



                                                                                                           
                                        Total time elapsed: 16.286 seconds                                 
                                       7 GPU function calls in 0.076 seconds                               
                                      1 CPU function calls in 13.631 seconds                               
                                                                                                           
                                                       Stats                                               
                                                                                                           
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function              ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame.query       │ 0          │ 0.000       │ 0.000       │ 1          │ 13.631      │ 13.631      │
│ DataFrame.__getitem__ │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.apply          │ 1          │ 0.045       │ 0.045       │ 0          │ 0.000       │ 0.000       │
│ Series.sort_values    │ 1          │ 0.010       │ 0.010       │ 0          │ 0.000       │ 0.000       │
│ Series.reset_index    │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.head        │ 1          │ 0.002       │ 0.002       │ 0          │ 0.000       │ 0.000       │
│ DataFrame             │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__repr__    │ 1          │ 0.017       │ 0.017       │ 0          │ 0.000       │ 0.000       │
└───────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- DataFrame.query

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=395655;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

V rezultatu vidimo tabelo s posameznimi operacijami in na katerem tipu procesorja so se izvajale. Vidimo, da je večina operacij tekla na GPE, operacija `.query()` pa na CPE. Našli smo vzrok za počasno izvajanje!

Toda kako je to mogoče? Saj smo v prejšnji poizvedbi prav tako izvedli operacijo `.query()` in se je brez problema izvajala na GPE! Poskusimo še z vrstičnim profiliranjem, kjer bo orodje vrnilo čas izvedbe po posameznih vrsticah naše kode. To aktiviramo z ukazom ``%%cudf.pandas.line_profile``.

In [18]:
%%cudf.pandas.line_profile
(gdf
 .query("`label` == 0 & `user` == 'lost_dog'")['tweet']
 .apply(lambda x: x.upper())
 .sort_values(ascending=False)
 .reset_index()
 .head(25)
)

                                                                                                       
                                      Total time elapsed: 17.635 seconds                               
                                                                                                       
                                                     Stats                                             
                                                                                                       
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                        ┃ GPU TIME(s) ┃ CPU TIME(s)  ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 1        │     (gdf                                                    │ 0.025319029 │              │
│          │                                                             │             │              │
│ 2        │      .query("`label` == 0 & `user` == 'lost_dog'")['tweet'] │ 2.102279458 │ 13.780252921 │
│          │                                                             │             │              │
│ 3        │      .apply(lambda x: x.upper())                            │ 0.045705954 │              │
│          │                                                             │             │              │
│ 4        │      .sort_values(ascending=False)                          │ 0.000288684 │              │
│          │                                                             │             │              │
│ 5        │      .reset_index()                                         │ 0.001180595 │              │
│          │                                                             │             │              │
│ 6        │      .head(25)                                              │ 0.002340418 │              │
│          │                                                             │             │              │
└──────────┴─────────────────────────────────────────────────────────────┴─────────────┴──────────────┘

V rezultatu vidimo nekaj zanimivega. Operacija ``.query()`` se izvaja nekaj časa na GPE, nekaj časa pa na CPE. Premislite zakaj!

Kaj točno delamo v tem delu kode? Najprej navedemo pogoj za stolpec `label`, ki je številčnega tipa. Nato pa navedemo še pogoj za stolpec `tweet`, ki ima tip niza znakov. V [dokumentaciji cuDF](https://docs.rapids.ai/api/cudf/stable/user_guide/api_docs/api/cudf.dataframe.query/#PandasCompat-0) podrobno preverimo ali je to sploh podprto v cuDF. Izkaže se, da so v cuDF za operacijo ``.query()`` trenutno podprti le numerični, datumski in logični tipi, **ne pa tudi nizi znakov**.

Ali to pomeni, da te poizvedbe ne moremo izvesti na GPE? Odgovor je "**lahko jo izvedemo na GPE**", vendar moramo spremeniti pristop. V poizvedbi smo z uporabo ``.query()`` filtrirali podatke glede na izbrana pogoja. To lahko v knjižnici Pandas storimo tudi drugače! Preoblikujmo našo poizvedbo tako, da bo uporabljala drugačen način filtriranja in še enkrat zaženimo profiliranje.

In [19]:
%%cudf.pandas.line_profile
(gdf[(gdf['label'] == 0) & (gdf['user'] == 'lost_dog')]['tweet'] # uporabili smo direktno filtriranje z operatorjem [] namesto .query()
 .apply(lambda x: x.upper())
 .sort_values(ascending=False)
 .reset_index()
 .head(25)
)

                                                                                                                   
                                             Total time elapsed: 3.561 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 1        │     (gdf[(gdf['label'] == 0) & (gdf['user'] == 'lost_dog')]['tweet'] # … │ 2.186335766 │             │
│          │                                                                          │             │             │
│ 2        │      .apply(lambda x: x.upper())                                         │ 0.045553700 │             │
│          │                                                                          │             │             │
│ 3        │      .sort_values(ascending=False)                                       │ 0.000299516 │             │
│          │                                                                          │             │             │
│ 4        │      .reset_index()                                                      │ 0.001193770 │             │
│          │                                                                          │             │             │
│ 5        │      .head(25)                                                           │ 0.002458540 │             │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

Zdaj poizvedba teče bistveno hitreje kot prej, še vedno pa je počasnejša od izvedbe brez pospeševanja. To je zato, ker se določene operacije bolj splačajo izvajati na CPE, spet druge pa se bolj splačajo izvajati na GPE. Med slednje spadajo razne računske operacije nad številčnimi tipi. Operacije za filtriranje podatkovnih zbirk še zaenkrat niso optimalno podprte v cuDF. Vredno je razmisliti, ali je to sploh smiselno izvajati na GPE, saj gre za drug tip procesorske enote. Lekcija tukaj je, da je morda bolj smiselno te tipe obdelave izvesti na CPE, obdelave računskega tipa pa na GPE. Upoštevati je potrebno tudi, da z obdelavo podatkov na GPE pravzaprav prenašamo podatke v pomnilnik GPE, kar se odraža v dodatnih zakasnitvah zaradi prenosa podatkov.

Kljub vsemu je dobro poznati različne možnosti obdelave velepodatkov, ki so nam na voljo. V tem izobraževanju ste spoznali različne napredne pristope za obdelavo velepodatkov. Spoznali ste napredne funkcionalnosti knjižnic za obdelavo velepodatkov, možnosti optimizacije pri izbiri datotečnih formatov in možnosti obdelave velepodatkov na različnih procesnih enotah (CPE in GPE). Skozi primere ste lahko ugotovili, da ne obstaja le ena najboljša rešitev, temveč je najboljša rešitev situacijska in odvisna od naših potreb. Prav tako je odvisna od tega kakšno strojno opremo imamo na voljo. S pridobljenim znanjem boste pripravljeni na reševanje izzivov obdelave velepodatkov ne glede na njihovo velikost in kompleksnost!